<a href="https://colab.research.google.com/github/mmfara/IF-EA/blob/main/Git_COMPAS_PIP_DIR%2BAL%2BROC_EO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data handling and visualization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# AIF360 fairness library
from aif360.datasets import StandardDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction, AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from sklearn.model_selection import train_test_split

data = pd.read_csv(COMPAS dataset)

X = data.drop(['two_year_recid'], axis =1)
y = data[['two_year_recid']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=101, shuffle =True, stratify = y)

dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

dataset_train = dataset_train.reset_index(drop=True)
dataset_test = dataset_test.reset_index(drop=True)

# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

# Define protected attribute names and privileged group
protected_attribute_names = ['race']
privileged_group = [{'race': 1}]
unprivileged_group = [{'race' : 0}]

dir = DisparateImpactRemover(repair_level=1, sensitive_attribute="race")

# Convert the training and test set to Binary label datasets
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=protected_attribute_names,
                             privileged_protected_attributes=privileged_group)

dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['two_year_recid'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=protected_attribute_names,
                                      privileged_protected_attributes=privileged_group)

# Fit and transform the training data
dir_dataset = dir.fit_transform(dataset_train_bld)

from aif360.algorithms.inprocessing import AdversarialDebiasing

sess = tf.Session()

# Use a different scope name if re-running the fitting process
debias_model = AdversarialDebiasing(privileged_groups=[{'race': 1}],
                                    unprivileged_groups=[{'race': 0}],
                                    scope_name='debias_model_unique_scope',  # Change the scope name for each new instance
                                    debias=True,
                                    sess=sess,
                                    seed = 101)

debias_model.fit(dir_dataset)

# Predict with the debias_model
predictions = debias_model.predict(dataset_test_bld)

from aif360.metrics import ClassificationMetric

# Assuming dataset_test_bld is the original test dataset with true labels
# and predictions is the dataset with your model's predictions
metric_predicted_dataset = ClassificationMetric(dataset_test_bld, predictions,
                              unprivileged_groups=unprivileged_group,
                              privileged_groups=privileged_group)


# Print metrics
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_predicted_dataset.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_predicted_dataset.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_predicted_dataset.mean_difference()))
print(f"Statistical Parity Difference: {metric_predicted_dataset.statistical_parity_difference()}")
print(f"Disparate Impact: {metric_predicted_dataset.disparate_impact()}")
print(f"Equal Opportunity Difference: {metric_predicted_dataset.equal_opportunity_difference()}")
print(f"Predictive Equality: {metric_predicted_dataset.false_positive_rate_difference()}")

#Implementing the ROC on AL output
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_group,
                                  privileged_groups=privileged_group, metric_name="Equal opportunity difference",
                                 low_class_thresh=0.3, high_class_thresh=0.8, num_class_thresh=100, num_ROC_margin=50,
                                 metric_ub=0.05, metric_lb=-0.05)

test_predictions = ROC.fit_predict(dataset_test_bld, predictions)

# Evaluate the classifier's performance with ROC intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_group,
    privileged_groups=privileged_group
)

print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.2f}".format(metric_with_fairness.accuracy()))
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

#Implementing the EO on AL output
# Initialize EqualizedOdds
eo = EqOddsPostprocessing(unprivileged_groups=unprivileged_group,
                                  privileged_groups=privileged_group, seed = 101)

# Fit the EqualizedOdds model
test_predictions = eo.fit_predict(dataset_test_bld, predictions)

# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_group,
    privileged_groups=privileged_group
)

print("Performance with fairness intervention (Equalized Odds):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())



